In [49]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import random
from autogluon.tabular import TabularPredictor
from sklearn.model_selection._split import _BaseKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


## Import the datasets

In [50]:
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

x_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
x_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
x_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

x_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
x_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
x_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

x_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
x_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
x_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

## Merge x_train observed and x_train_estimated

In [51]:
x_train_a = pd.concat([x_train_observed_a,x_train_estimated_a])
x_train_b = pd.concat([x_train_observed_b,x_train_estimated_b])
x_train_c = pd.concat([x_train_observed_c,x_train_estimated_c])

In [52]:
x_train_a

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,date_calc
0,2019-06-02 22:00:00,7.7,1.230,1744.900024,0.0,0.0,1744.900024,0.0,280.299988,0.0,0.0,0.0,0.0,98.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.799988,0.0,0.0,994.200012,1000.299988,0.0,0.0,73.099998,1006.299988,NaN,0.0,0.0,-0.0,0.1,342.834015,-3.202,0.0,285.899994,100.000000,39640.101562,3.7,-3.6,-0.8,-0.0,NaT
1,2019-06-02 22:15:00,7.7,1.229,1734.000000,0.0,0.0,1734.000000,0.0,280.299988,0.0,0.0,0.0,0.0,99.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.500000,0.0,0.0,993.900024,999.900024,0.0,0.0,72.199997,1006.000000,NaN,0.0,0.0,-0.0,0.2,346.294006,-3.650,0.0,286.100006,100.000000,40123.898438,3.6,-3.6,-0.6,-0.0,NaT
2,2019-06-02 22:30:00,7.7,1.228,1723.500000,0.0,0.0,1723.500000,0.0,280.299988,0.0,0.0,0.0,0.0,99.199997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.099976,0.0,0.0,993.599976,999.599976,0.0,0.0,71.199997,1005.599976,NaN,0.0,0.0,-0.0,0.2,349.768005,-3.998,0.0,286.299988,100.000000,40628.300781,3.6,-3.6,-0.4,-0.0,NaT
3,2019-06-02 22:45:00,7.7,1.226,1713.400024,0.0,0.0,1713.400024,0.0,280.299988,0.0,0.0,0.0,0.0,99.400002,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1005.799988,0.0,0.0,993.299988,999.299988,0.0,0.0,70.199997,1005.299988,NaN,0.0,0.0,-0.0,0.2,353.251007,-4.247,0.0,286.600006,100.000000,41153.601562,3.5,-3.5,-0.2,-0.0,NaT
4,2019-06-02 23:00:00,7.7,1.225,1703.599976,0.0,0.0,1703.599976,0.0,280.299988,0.0,0.0,0.0,0.0,99.599998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1005.500000,0.0,0.0,993.000000,999.000000,0.0,0.0,69.199997,1005.000000,NaN,0.0,0.0,-0.0,0.2,356.742004,-4.393,0.0,286.799988,100.000000,41699.898438,3.5,-3.5,0.0,-0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17571,2023-04-30 22:45:00,4.5,1.281,1173.900024,0.0,0.0,536.500000,0.0,272.299988,0.0,0.0,0.0,0.0,99.599998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.299988,0.0,0.0,1001.099976,1007.400024,0.0,0.0,79.599998,1013.700012,NaN,0.0,0.0,-0.0,0.1,352.920013,-11.731,0.1,274.200012,99.900002,11629.299805,3.9,2.9,2.5,-0.0,2023-04-29 07:00:05
17572,2023-04-30 23:00:00,4.5,1.281,1054.199951,0.0,0.0,542.400024,0.0,272.200012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.299988,0.0,0.0,1001.099976,1007.299988,0.0,0.0,80.000000,1013.599976,NaN,0.0,0.0,-0.0,0.1,356.634003,-11.884,0.1,274.200012,100.000000,9923.200195,3.7,2.8,2.4,-0.0,2023-04-29 07:00:05
17573,2023-04-30 23:15:00,4.5,1.281,1435.800049,0.0,0.0,531.799988,0.0,272.299988,0.0,0.0,0.0,0.0,99.400002,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.099976,0.0,0.0,1000.900024,1007.200012,0.0,0.0,79.900002,1013.500000,NaN,0.0,0.0,-0.0,0.1,0.352000,-11.928,0.1,274.200012,99.900002,11230.799805,3.7,2.7,2.5,-0.0,2023-04-29 07:00:05
17574,2023-04-30 23:30:00,4.5,1.281,1817.400024,0.0,0.0,521.200012,0.0,272.299988,0.0,0.0,0.0,0.0,98.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.000000,0.0,0.0,1000.799988,1007.000000,0.0,0.0,79.800003,1013.299988,NaN,0.0,0.0,-0.0,0.0,4.070000,-11.864,0.1,274.200012,99.599998,12526.099609,3.8,2.7,2.6,-0.0,2023-04-29 07:00:05


## Part 1: Data Preprocessing

1. Remove NaN columns
2. Aggregate data from 15 minute to hourly intervals:
   - Method 1: Take the mean over all four 15minutes recording, resulting in hourly measurements.
   - Method 2: Create a separate column for each 15-minute value, effectively quadrupling the number of columns.
3. Remove rows in train with NaN values in pv measurement
4. Handle consecutive pv measurments
5. Remove rows that have timestamps that are not present in both x and y
6. Remove rows with NaN values in all columns


### 1. Remove NaN columns

In [53]:
#Remove snow:density column as well as rows with only NaN values
def remove_nan_cols(x,remove_cols):
    df = x.copy()
    df = df.drop(columns = remove_cols ) #Should we include 'cloud_base_agl:m' and ceiling_height_agl:m ['snow_density:kgm3']
    return df

In [54]:
#For Catboost
x_train_a1 = remove_nan_cols(x_train_a,['snow_density:kgm3'])
x_train_b1 = remove_nan_cols(x_train_b,['snow_density:kgm3'])
x_train_c1 = remove_nan_cols(x_train_c,['snow_density:kgm3'])

x_test_a1 = remove_nan_cols(x_test_estimated_a,['snow_density:kgm3'])
x_test_b1 = remove_nan_cols(x_test_estimated_b,['snow_density:kgm3'])
x_test_c1 = remove_nan_cols(x_test_estimated_c,['snow_density:kgm3'])

In [55]:
#For Autogluon
x_train_a2 =  remove_nan_cols(x_train_a,['snow_density:kgm3', 'cloud_base_agl:m'])
x_train_b2 =  remove_nan_cols(x_train_b,['snow_density:kgm3', 'cloud_base_agl:m'])
x_train_c2 =  remove_nan_cols(x_train_c,['snow_density:kgm3', 'cloud_base_agl:m'])

x_test_a2 =  remove_nan_cols(x_test_estimated_a,['snow_density:kgm3', 'cloud_base_agl:m'])
x_test_b2 =  remove_nan_cols(x_test_estimated_b,['snow_density:kgm3', 'cloud_base_agl:m'])
x_test_c2 =  remove_nan_cols(x_test_estimated_c,['snow_density:kgm3', 'cloud_base_agl:m'])

### 2. Transform data to hourly

**Method 1: Take the mean over all four 15minutes recording, resulting in hourly measurements.**

In [56]:
def resample_to_hourly(x): 
    df = x.copy()
    df.set_index('date_forecast', inplace=True)
    
    df_hourly_date_calc = df.resample('H')['date_calc'].first()
    # Aggregating by averaging over quartarly measurements
    df_hourly = df.resample('H').mean()
    df_hourly['date_calc'] = df_hourly_date_calc
    df_hourly.reset_index(inplace=True)
    
    return df_hourly

x_train_a_hourly = resample_to_hourly(x_train_a1)
x_train_b_hourly = resample_to_hourly(x_train_b1)
x_train_c_hourly = resample_to_hourly(x_train_c1)

x_test_a_hourly = resample_to_hourly(x_test_a1)
x_test_b_hourly = resample_to_hourly(x_test_b1)
x_test_c_hourly = resample_to_hourly(x_test_c1)

In [57]:
x_train_a_hourly

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,date_calc
0,2019-06-02 22:00:00,7.700,1.22825,1728.949951,0.000000,0.000,1728.949951,0.0,280.299988,0.000,0.000000,0.00,0.000000,99.074997,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1006.299988,0.0,0.0,993.750000,999.775024,0.0,0.000,71.674995,1005.799988,0.0,0.0,0.0,0.175,348.036743,-3.77425,0.000,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.0,NaT
1,2019-06-02 23:00:00,7.700,1.22350,1689.824951,0.000000,0.000,1689.824951,0.0,280.299988,0.000,0.000000,0.00,0.000000,99.750000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1005.200012,0.0,0.0,992.674988,998.650024,0.0,0.025,68.000000,1004.650024,0.0,0.0,0.0,0.200,91.980751,-4.35725,0.000,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.0,NaT
2,2019-06-03 00:00:00,7.875,1.21975,1563.224976,0.000000,0.000,1563.224976,0.0,280.649994,0.000,0.000000,0.00,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1004.525024,0.0,0.0,992.000000,997.974976,0.0,0.100,67.949997,1003.950012,0.0,0.0,0.0,0.400,14.934750,-3.30950,0.000,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.0,NaT
3,2019-06-03 01:00:00,8.425,1.21800,1283.425049,208.649994,0.750,1283.425049,0.0,281.674988,0.300,526.775024,0.00,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.25,1.00,1004.025024,0.0,0.0,991.500000,997.449951,0.0,0.125,73.875000,1003.449951,0.0,0.0,0.0,0.550,28.630251,-0.82250,0.000,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.0,NaT
4,2019-06-03 02:00:00,8.950,1.21800,1003.500000,32468.150391,23.100,1003.500000,0.0,282.500000,11.975,22068.949219,0.15,282.975006,84.875000,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1003.099976,0.0,0.0,990.550049,996.500000,0.0,0.100,79.925003,1002.500000,0.0,0.0,0.0,0.250,41.997501,3.05125,0.000,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34269,2023-04-30 19:00:00,4.550,1.27650,1674.199951,84464.945312,4.225,542.700012,0.0,272.424988,2.825,56431.050781,0.10,13230.649414,96.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.50,0.75,1014.750000,0.0,0.0,1001.574951,1007.799988,0.0,0.000,74.625000,1014.049988,0.0,0.0,0.0,0.000,304.939240,-0.18050,0.000,275.024994,96.699997,23417.074219,5.175,4.800,1.925,0.0,2023-04-29 07:00:05
34270,2023-04-30 20:00:00,4.500,1.27975,1762.400024,2270.875000,0.000,546.400024,0.0,272.299988,0.000,5067.875000,0.00,176.350006,94.224998,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1014.700012,0.0,0.0,1001.549988,1007.799988,0.0,0.000,76.875000,1014.049988,0.0,0.0,0.0,0.000,318.625763,-5.17600,0.000,274.650024,94.525002,21084.050781,4.650,4.025,2.300,0.0,2023-04-29 07:00:05
34271,2023-04-30 21:00:00,4.500,1.28100,1696.650024,0.000000,0.000,548.349976,0.0,272.299988,0.000,0.000000,0.00,0.000000,94.324997,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1014.549988,0.0,0.0,1001.400024,1007.674988,0.0,0.000,77.775002,1013.925049,0.0,0.0,0.0,0.000,332.785736,-8.95075,0.000,274.524994,95.675003,20792.500000,4.450,3.575,2.600,0.0,2023-04-29 07:00:05
34272,2023-04-30 22:00:00,4.500,1.28100,1353.400024,0.000000,0.000,527.775024,0.0,272.299988,0.000,0.000000,0.00,0.000000,97.775002,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1014.400024,

In [58]:
#use only rows in test that are given in the test csv
test = pd.read_csv('test.csv')
pred_time_stamps = test['time'].unique()
x_test_a1 = x_test_a_hourly[x_test_a_hourly['date_forecast'].isin(pred_time_stamps)]
x_test_b1 = x_test_b_hourly[x_test_b_hourly['date_forecast'].isin(pred_time_stamps)]
x_test_c1 = x_test_c_hourly[x_test_c_hourly['date_forecast'].isin(pred_time_stamps)]

**Method 2: Create a separate column for each 15-minute value, quadrupling the number of columns.**

In [59]:
def resample_to_hourly_quarters(x, date_column='date_forecast', exclude_column='date_calc'):
    df = x.copy()
    # Ensure the date column is in datetime format and set as the index
    df[date_column] = pd.to_datetime(df[date_column])
    df.set_index(date_column, inplace=True)
    
    # Separate the column to exclude from the resampling
    excluded_data = df[[exclude_column]].resample('H').first()  

    # Remove the excluded column from df before pivoting, in this case date_calc
    df = df.drop(columns=[exclude_column])

    # Add a column for the 15-minute period within the hour
    df['quarter'] = df.index.minute // 15  # Use floor division to get the quarter number (0, 1, 2, 3)

    # Pivot the table. For each feature, create a new column for each 15-minute period.
    df_pivot = df.pivot_table(index=df.index.floor('H'),
                              columns='quarter',
                              aggfunc='first')  # We use 'first' because each quarter should be unique
    
    # Flatten the multi-level column index
    df_pivot.columns = ['{}_Q{}'.format(feature, quarter) for feature, quarter in df_pivot.columns]

    # Reset the index to be able to merge on the date_column
    df_pivot.reset_index(inplace=True)
    excluded_data.reset_index(inplace=True)

    # Merge back the excluded column
    df_hourly = pd.merge(excluded_data, df_pivot, on=date_column)

    return df_hourly

# Make sure to pass the column name that contains the datetime information
x_train_a_hourly2 = resample_to_hourly_quarters(x_train_a2, date_column='date_forecast')
x_train_b_hourly2 = resample_to_hourly_quarters(x_train_b2, date_column='date_forecast')
x_train_c_hourly2 = resample_to_hourly_quarters(x_train_c2, date_column='date_forecast')

x_test_a_hourly2 = resample_to_hourly_quarters(x_test_a2, date_column='date_forecast')
x_test_b_hourly2 = resample_to_hourly_quarters(x_test_b2, date_column='date_forecast')
x_test_c_hourly2 = resample_to_hourly_quarters(x_test_b2, date_column='date_forecast')

### 3. Remove rows in train with NaN values in pv measurement

In [60]:

# Identify the indices of the rows with NaN values in the 'pv_measurement' column
nan_indices_a = train_a[train_a['pv_measurement'].isna()].index
nan_indices_b = train_b[train_b['pv_measurement'].isna()].index
nan_indices_c = train_c[train_c['pv_measurement'].isna()].index

# Drop these indices from y_train
train_a = train_a.drop(nan_indices_a).reset_index(drop = True)
train_b = train_b.drop(nan_indices_b).reset_index(drop = True)
train_c = train_c.drop(nan_indices_c).reset_index(drop = True)



### 4. Handle consecutive pv measurments

In [61]:
# Filters out rows from a DataFrame where the 'pv_measurement' column has consecutive zero-values beyond a specified threshold. 
# Also removes all rows, except the first, where 'pv_measurement' is identical for two or more rows. 

def remove_constant_intervals(df: pd.DataFrame, threshold: int = 24) -> pd.DataFrame:
    """
    Filters out rows from a DataFrame where the 'pv_measurement' column has:
    1. Consecutive zero values beyond a specified threshold.
    2. Consecutive non-zero identical values, keeping only the first in each sequence.

    Parameters:
    - df: Input DataFrame to be processed.
    - threshold: Maximum allowable number of consecutive zero values. Rows in streaks
      beyond this threshold will be removed.

    Returns:
    - Filtered DataFrame.
    """
    # Create a mask to identify where the current 'pv_measurement'-row is different from the previous one
    is_different = df['pv_measurement'] != df['pv_measurement'].shift()
    
    # Additional mask to identify non-zero values
    is_nonzero = df['pv_measurement'] != 0

    # Use cumsum to generate unique group identifiers for each streak of identical measurements
    groups = is_different.cumsum()

    # Count the number of entries in each group
    df['group_counts'] = df.groupby(groups)['pv_measurement'].transform('count')

    # Identify groups of zeros that exceed the specified threshold
    to_remove_zeros = (df['group_counts'] > threshold) & ~is_nonzero

    # Identify the first entry in groups of consecutive non-zero identical values
    to_keep_nonzero = (df['pv_measurement'].shift() != df['pv_measurement']) | is_different | ~is_nonzero

    # Combine conditions
    to_remove = to_remove_zeros | ~to_keep_nonzero

    # Drop the temporary 'group_counts' column
    df = df.drop(columns=['group_counts'])

    # Filter out the rows that do not meet the conditions
    filtered_df = df[~to_remove]
    return filtered_df

In [62]:
#Remove rows in groups of constant values, where duration of constant measurements is > 1 day (24 hours)
train_a = remove_constant_intervals(train_a,24)
train_b = remove_constant_intervals(train_b,24)
train_c = remove_constant_intervals(train_c,24)

### 5. Remove rows that have timestamps that are not present in both x and y

In [63]:
# Remove all rows with date-time values that are not present in both x and y in order to synchronize x and its labels. 
def remove_non_synchronous_rows(x_train, y_train, x_date_column='date_forecast', y_date_column='time'):
    # Convert date columns to datetime format for easier comparison
    x_train[x_date_column] = pd.to_datetime(x_train[x_date_column])
    y_train[y_date_column] = pd.to_datetime(y_train[y_date_column])
    
    # Find common dates
    common_dates = x_train[x_date_column][x_train[x_date_column].isin(y_train[y_date_column])]
    
    # Filter both datasets based on common dates
    x_train_synced = x_train.loc[x_train[x_date_column].isin(common_dates)]
    y_train_synced = y_train.loc[y_train[y_date_column].isin(common_dates)]
    
    return x_train_synced, y_train_synced

# Remove the rows with date and time that only shows up in one of the sets
x_train_a1, train_a = remove_non_synchronous_rows(x_train_a_hourly, train_a)
x_train_b1, train_b = remove_non_synchronous_rows(x_train_b_hourly, train_b)
x_train_c1, train_c = remove_non_synchronous_rows(x_train_c_hourly, train_c)

# Remove the rows with date and time that only shows up in one of the sets
x_train_a2, train_a = remove_non_synchronous_rows(x_train_a_hourly2, train_a)
x_train_b2, train_b = remove_non_synchronous_rows(x_train_b_hourly2, train_b)
x_train_c2, train_c = remove_non_synchronous_rows(x_train_c_hourly2, train_c)

### 6. Remove rows with NaN values in all columns

In [64]:
#For Catboost
x_train_a1 = x_train_a1.dropna(subset=['diffuse_rad:W'])
x_train_b1 = x_train_b1.dropna(subset=['diffuse_rad:W'])
x_train_c1 = x_train_c1.dropna(subset=['diffuse_rad:W'])

x_test_a1 = x_test_a1.dropna(subset=['diffuse_rad:W'])
x_test_b1 = x_test_b1.dropna(subset=['diffuse_rad:W'])
x_test_c1 = x_test_c1.dropna(subset=['diffuse_rad:W'])

In [65]:
x_train_a2.head()

,date_forecast,date_calc,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,ceiling_height_agl:m_Q1,ceiling_height_agl:m_Q2,ceiling_height_agl:m_Q3,clear_sky_energy_1h:J_Q0,clear_sky_energy_1h:J_Q1,clear_sky_energy_1h:J_Q2,clear_sky_energy_1h:J_Q3,clear_sky_rad:W_Q0,clear_sky_rad:W_Q1,clear_sky_rad:W_Q2,clear_sky_rad:W_Q3,dew_or_rime:idx_Q0,dew_or_rime:idx_Q1,dew_or_rime:idx_Q2,dew_or_rime:idx_Q3,dew_point_2m:K_Q0,dew_point_2m:K_Q1,dew_point_2m:K_Q2,dew_point_2m:K_Q3,diffuse_rad:W_Q0,diffuse_rad:W_Q1,diffuse_rad:W_Q2,diffuse_rad:W_Q3,diffuse_rad_1h:J_Q0,diffuse_rad_1h:J_Q1,diffuse_rad_1h:J_Q2,diffuse_rad_1h:J_Q3,direct_rad:W_Q0,direct_rad:W_Q1,direct_rad:W_Q2,direct_rad:W_Q3,direct_rad_1h:J_Q0,direct_rad_1h:J_Q1,direct_rad_1h:J_Q2,direct_rad_1h:J_Q3,effective_cloud_cover:p_Q0,effective_cloud_cover:p_Q1,effective_cloud_cover:p_Q2,effective_cloud_cover:p_Q3,elevation:m_Q0,elevation:m_Q1,elevation:m_Q2,elevation:m_Q3,fresh_snow_12h:cm_Q0,fresh_snow_12h:cm_Q1,fresh_snow_12h:cm_Q2,fresh_snow_12h:cm_Q3,fresh_snow_1h:cm_Q0,fresh_snow_1h:cm_Q1,fresh_snow_1h:cm_Q2,fresh_snow_1h:cm_Q3,fresh_snow_24h:cm_Q0,fresh_snow_24h:cm_Q1,fresh_snow_24h:cm_Q2,fresh_snow_24h:cm_Q3,fresh_snow_3h:cm_Q0,fresh_snow_3h:cm_Q1,fresh_snow_3h:cm_Q2,fresh_snow_3h:cm_Q3,fresh_snow_6h:cm_Q0,fresh_snow_6h:cm_Q1,fresh_snow_6h:cm_Q2,fresh_snow_6h:cm_Q3,is_day:idx_Q0,is_day:idx_Q1,is_day:idx_Q2,is_day:idx_Q3,is_in_shadow:idx_Q0,is_in_shadow:idx_Q1,is_in_shadow:idx_Q2,is_in_shadow:idx_Q3,msl_pressure:hPa_Q0,msl_pressure:hPa_Q1,msl_pressure:hPa_Q2,msl_pressure:hPa_Q3,precip_5min:mm_Q0,precip_5min:mm_Q1,precip_5min:mm_Q2,precip_5min:mm_Q3,precip_type_5min:idx_Q0,precip_type_5min:idx_Q1,precip_type_5min:idx_Q2,precip_type_5min:idx_Q3,pressure_100m:hPa_Q0,pressure_100m:hPa_Q1,pressure_100m:hPa_Q2,pressure_100m:hPa_Q3,pressure_50m:hPa_Q0,pressure_50m:hPa_Q1,pressure_50m:hPa_Q2,pressure_50m:hPa_Q3,prob_rime:p_Q0,prob_rime:p_Q1,prob_rime:p_Q2,prob_rime:p_Q3,rain_water:kgm2_Q0,rain_water:kgm2_Q1,rain_water:kgm2_Q2,rain_water:kgm2_Q3,relative_humidity_1000hPa:p_Q0,relative_humidity_1000hPa:p_Q1,relative_humidity_1000hPa:p_Q2,relative_humidity_1000hPa:p_Q3,sfc_pressure:hPa_Q0,sfc_pressure:hPa_Q1,sfc_pressure:hPa_Q2,sfc_pressure:hPa_Q3,snow_depth:cm_Q0,snow_depth:cm_Q1,snow_depth:cm_Q2,snow_depth:cm_Q3,snow_drift:idx_Q0,snow_drift:idx_Q1,snow_drift:idx_Q2,snow_drift:idx_Q3,snow_melt_10min:mm_Q0,snow_melt_10min:mm_Q1,snow_melt_10min:mm_Q2,snow_melt_10min:mm_Q3,snow_water:kgm2_Q0,snow_water:kgm2_Q1,snow_water:kgm2_Q2,snow_water:kgm2_Q3,sun_azimuth:d_Q0,sun_azimuth:d_Q1,sun_azimuth:d_Q2,sun_azimuth:d_Q3,sun_elevation:d_Q0,sun_elevation:d_Q1,sun_elevation:d_Q2,sun_elevation:d_Q3,super_cooled_liquid_water:kgm2_Q0,super_cooled_liquid_water:kgm2_Q1,super_cooled_liquid_water:kgm2_Q2,super_cooled_liquid_water:kgm2_Q3,t_1000hPa:K_Q0,t_1000hPa:K_Q1,t_1000hPa:K_Q2,t_1000hPa:K_Q3,total_cloud_cover:p_Q0,total_cloud_cover:p_Q1,total_cloud_cover:p_Q2,total_cloud_cover:p_Q3,visibility:m_Q0,visibility:m_Q1,visibility:m_Q2,visibility:m_Q3,wind_speed_10m:ms_Q0,wind_speed_10m:ms_Q1,wind_speed_10m:ms_Q2,wind_speed_10m:ms_Q3,wind_speed_u_10m:ms_Q0,wind_speed_u_10m:ms_Q1,wind_speed_u_10m:ms_Q2,wind_speed_u_10m:ms_Q3,wind_speed_v_10m:ms_Q0,wind_speed_v_10m:ms_Q1,wind_speed_v_10m:ms_Q2,wind_speed_v_10m:ms_Q3,wind_speed_w_1000hPa:ms_Q0,wind_speed_w_1000hPa:ms_Q1,wind_speed_w_1000hPa:ms_Q2,wind_speed_w_1000hPa:ms_Q3
0,2019-06-02 22:00:00,NaT,7.7,7.7,7.7,7.7,1.230,1.229,1.228,1.226,1744.900024,1734.000000,1723.500000,1713.400024,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,280.299988,280.299988,280.299988,280.299988,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,98.699997,99.000000,99.199997,99.400002,6.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [66]:
#For Autogluon
x_train_a2 = x_train_a2.dropna(subset=['diffuse_rad:W_Q1'])
x_train_b2 = x_train_b2.dropna(subset=['diffuse_rad:W_Q1'])
x_train_c2 = x_train_c2.dropna(subset=['diffuse_rad:W_Q1'])

x_test_a2 = x_test_a_hourly2.dropna(subset=['diffuse_rad:W_Q1'])
x_test_b2 = x_test_b_hourly2.dropna(subset=['diffuse_rad:W_Q1'])
x_test_c2 = x_test_c_hourly2.dropna(subset=['diffuse_rad:W_Q1'])

## Part 2: Feature Engineering 
1. Add time features (hour, day, month, year) + binary observed column
2. Add cyclical features
3. Add direct_rad x sun_elevation feature
4. Remove 'date_forecast' from test data

### 1. Add time features: hour, day, month, year

In [67]:
# Extracts year, month, day, and hour features from a given datetime column
def extract_date_features(X):
    df = X.copy()
    # Extract features
    df['year'] = df['date_forecast'].dt.year
    df['month'] = df['date_forecast'].dt.month
    df['day'] = df['date_forecast'].dt.day
    df['hour'] = df['date_forecast'].dt.hour
    
    df['observed'] = (df['date_calc'].isna()).astype(int)
    df['observed'] = df['observed'].astype(str)
    
    
    df = df.drop(columns = ['date_calc'])
    
    return df

In [68]:
x_train_a1 = extract_date_features(x_train_a1)
x_train_b1 = extract_date_features(x_train_b1)
x_train_c1 = extract_date_features(x_train_c1)

x_test_a1 = extract_date_features(x_test_a1)
x_test_b1 = extract_date_features(x_test_b1)
x_test_c1 = extract_date_features(x_test_c1)


In [69]:
x_train_a2 = extract_date_features(x_train_a2)
x_train_b2 = extract_date_features(x_train_b2)
x_train_c2 = extract_date_features(x_train_c2)

x_test_a2 = extract_date_features(x_test_a2)
x_test_b2 = extract_date_features(x_test_b2)
x_test_c2 = extract_date_features(x_test_c2)

### 2. Add cyclical features

In [70]:
# Creating cyclical features for hour of the day and month of the year
def add_cyclic(x_train):
    train_data = x_train.copy()
   
    train_data['hour_sin'] = np.sin(2 * np.pi * train_data['hour'] / 24)
    train_data['hour_cos'] = np.cos(2 * np.pi * train_data['hour'] / 24)
    train_data['month_sin'] = np.sin(2 * np.pi * (train_data['month']-1) / 12)
    train_data['month_cos'] = np.cos(2 * np.pi * (train_data['month']-1) / 12)
    
    #train_data.drop(columns = ['hour','month'],inplace = True)
    return train_data

x_train_a1 = add_cyclic(x_train_a1)
x_train_b1 = add_cyclic(x_train_b1)
x_train_c1 = add_cyclic(x_train_c1)

x_test_a1 = add_cyclic(x_test_a1)
x_test_b1 = add_cyclic(x_test_b1)
x_test_c1 = add_cyclic(x_test_c1)

x_train_a2 = add_cyclic(x_train_a2)
x_train_b2 = add_cyclic(x_train_b2)
x_train_c2 = add_cyclic(x_train_c2)

x_test_a2 = add_cyclic(x_test_a2)
x_test_b2 = add_cyclic(x_test_b2)
x_test_c2 = add_cyclic(x_test_c2)

### 3. Add direct_rad x sun_elevation feature for Autogluon

In [71]:
def add_interaction_feature(x):
    data = x.copy()
    # Create interaction term between 'direct_rad:W' and 'sun_elevation:d'
    data['direct_rad_x_sun_elevation_Q0'] = data['direct_rad:W_Q0'] * data['sun_elevation:d_Q0']
    data['direct_rad_x_sun_elevation_Q1'] = data['direct_rad:W_Q1'] * data['sun_elevation:d_Q1']
    data['direct_rad_x_sun_elevation_Q2'] = data['direct_rad:W_Q2'] * data['sun_elevation:d_Q2']
    data['direct_rad_x_sun_elevation_Q3'] = data['direct_rad:W_Q3'] * data['sun_elevation:d_Q3']
    return data


x_train_a2 = add_interaction_feature(x_train_a2)
x_train_b2 = add_interaction_feature(x_train_b2)
x_train_c2 = add_interaction_feature(x_train_c2)

x_test_a2 = add_interaction_feature(x_test_a2)
x_test_b2 = add_interaction_feature(x_test_b2)
x_test_c2 = add_interaction_feature(x_test_c2)

### 4. Remove 'date_forecast' from test data

In [72]:
x_test_a1=x_test_a1.drop(columns = ['date_forecast'])
x_test_b1=x_test_b1.drop(columns = ['date_forecast'])
x_test_c1=x_test_c1.drop(columns = ['date_forecast'])

x_test_a2=x_test_a2.drop(columns = ['date_forecast'])
x_test_b2=x_test_b2.drop(columns = ['date_forecast'])
x_test_c2=x_test_c2.drop(columns = ['date_forecast'])

,index,date_forecast,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,ceiling_height_agl:m_Q1,ceiling_height_agl:m_Q2,ceiling_height_agl:m_Q3,clear_sky_energy_1h:J_Q0,clear_sky_energy_1h:J_Q1,clear_sky_energy_1h:J_Q2,clear_sky_energy_1h:J_Q3,clear_sky_rad:W_Q0,clear_sky_rad:W_Q1,clear_sky_rad:W_Q2,clear_sky_rad:W_Q3,dew_or_rime:idx_Q0,dew_or_rime:idx_Q1,dew_or_rime:idx_Q2,dew_or_rime:idx_Q3,dew_point_2m:K_Q0,dew_point_2m:K_Q1,dew_point_2m:K_Q2,dew_point_2m:K_Q3,diffuse_rad:W_Q0,diffuse_rad:W_Q1,diffuse_rad:W_Q2,diffuse_rad:W_Q3,diffuse_rad_1h:J_Q0,diffuse_rad_1h:J_Q1,diffuse_rad_1h:J_Q2,diffuse_rad_1h:J_Q3,direct_rad:W_Q0,direct_rad:W_Q1,direct_rad:W_Q2,direct_rad:W_Q3,direct_rad_1h:J_Q0,direct_rad_1h:J_Q1,direct_rad_1h:J_Q2,direct_rad_1h:J_Q3,effective_cloud_cover:p_Q0,effective_cloud_cover:p_Q1,effective_cloud_cover:p_Q2,effective_cloud_cover:p_Q3,elevation:m_Q0,elevation:m_Q1,elevation:m_Q2,elevation:m_Q3,fresh_snow_12h:cm_Q0,fresh_snow_12h:cm_Q1,fresh_snow_12h:cm_Q2,fresh_snow_12h:cm_Q3,fresh_snow_1h:cm_Q0,fresh_snow_1h:cm_Q1,fresh_snow_1h:cm_Q2,fresh_snow_1h:cm_Q3,fresh_snow_24h:cm_Q0,fresh_snow_24h:cm_Q1,fresh_snow_24h:cm_Q2,fresh_snow_24h:cm_Q3,fresh_snow_3h:cm_Q0,fresh_snow_3h:cm_Q1,fresh_snow_3h:cm_Q2,fresh_snow_3h:cm_Q3,fresh_snow_6h:cm_Q0,fresh_snow_6h:cm_Q1,fresh_snow_6h:cm_Q2,fresh_snow_6h:cm_Q3,is_day:idx_Q0,is_day:idx_Q1,is_day:idx_Q2,is_day:idx_Q3,is_in_shadow:idx_Q0,is_in_shadow:idx_Q1,is_in_shadow:idx_Q2,is_in_shadow:idx_Q3,msl_pressure:hPa_Q0,msl_pressure:hPa_Q1,msl_pressure:hPa_Q2,msl_pressure:hPa_Q3,precip_5min:mm_Q0,precip_5min:mm_Q1,precip_5min:mm_Q2,precip_5min:mm_Q3,precip_type_5min:idx_Q0,precip_type_5min:idx_Q1,precip_type_5min:idx_Q2,precip_type_5min:idx_Q3,pressure_100m:hPa_Q0,pressure_100m:hPa_Q1,pressure_100m:hPa_Q2,pressure_100m:hPa_Q3,pressure_50m:hPa_Q0,pressure_50m:hPa_Q1,pressure_50m:hPa_Q2,pressure_50m:hPa_Q3,prob_rime:p_Q0,prob_rime:p_Q1,prob_rime:p_Q2,prob_rime:p_Q3,rain_water:kgm2_Q0,rain_water:kgm2_Q1,rain_water:kgm2_Q2,rain_water:kgm2_Q3,relative_humidity_1000hPa:p_Q0,relative_humidity_1000hPa:p_Q1,relative_humidity_1000hPa:p_Q2,relative_humidity_1000hPa:p_Q3,sfc_pressure:hPa_Q0,sfc_pressure:hPa_Q1,sfc_pressure:hPa_Q2,sfc_pressure:hPa_Q3,snow_depth:cm_Q0,snow_depth:cm_Q1,snow_depth:cm_Q2,snow_depth:cm_Q3,snow_drift:idx_Q0,snow_drift:idx_Q1,snow_drift:idx_Q2,snow_drift:idx_Q3,snow_melt_10min:mm_Q0,snow_melt_10min:mm_Q1,snow_melt_10min:mm_Q2,snow_melt_10min:mm_Q3,snow_water:kgm2_Q0,snow_water:kgm2_Q1,snow_water:kgm2_Q2,snow_water:kgm2_Q3,sun_azimuth:d_Q0,sun_azimuth:d_Q1,sun_azimuth:d_Q2,sun_azimuth:d_Q3,sun_elevation:d_Q0,sun_elevation:d_Q1,sun_elevation:d_Q2,sun_elevation:d_Q3,super_cooled_liquid_water:kgm2_Q0,super_cooled_liquid_water:kgm2_Q1,super_cooled_liquid_water:kgm2_Q2,super_cooled_liquid_water:kgm2_Q3,t_1000hPa:K_Q0,t_1000hPa:K_Q1,t_1000hPa:K_Q2,t_1000hPa:K_Q3,total_cloud_cover:p_Q0,total_cloud_cover:p_Q1,total_cloud_cover:p_Q2,total_cloud_cover:p_Q3,visibility:m_Q0,visibility:m_Q1,visibility:m_Q2,visibility:m_Q3,wind_speed_10m:ms_Q0,wind_speed_10m:ms_Q1,wind_speed_10m:ms_Q2,wind_speed_10m:ms_Q3,wind_speed_u_10m:ms_Q0,wind_speed_u_10m:ms_Q1,wind_speed_u_10m:ms_Q2,wind_speed_u_10m:ms_Q3,wind_speed_v_10m:ms_Q0,wind_speed_v_10m:ms_Q1,wind_speed_v_10m:ms_Q2,wind_speed_v_10m:ms_Q3,wind_speed_w_1000hPa:ms_Q0,wind_speed_w_1000hPa:ms_Q1,wind_speed_w_1000hPa:ms_Q2,wind_speed_w_1000hPa:ms_Q3,year,month,day,hour,observed,hour_sin,hour_cos,month_sin,month_cos,direct_rad_x_sun_elevation_Q0,direct_rad_x_sun_elevation_Q1,direct_rad_x_sun_elevation_Q2,direct_rad_x_sun_elevation_Q3
0,0,2019-06-02 22:00:00,7.7,7.7,7.7,7.7,1.230,1.229,1.228,1.226,1744.900024,1734.000000,1723.500000,1713.400024,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,280.299988,280.299988,280.299988,280.299988,0.0,0.

## Part 2: Model Building 
1. Catboost Model
2. Autogluon Model

### 1. Catboost

In [73]:
#Merge x_train and train for training models
merged_a1 = pd.merge(x_train_a1, train_a, left_on='date_forecast', right_on='time', how='inner')
merged_b1 = pd.merge(x_train_b1, train_b, left_on='date_forecast', right_on='time', how='inner')
merged_c1 = pd.merge(x_train_c1, train_c, left_on='date_forecast', right_on='time', how='inner')

In [74]:
def build_catboost_multiple_seed(merged_df,x_test,number_of_models):
    merged_df = merged_df.drop(columns=['date_forecast', 'time'])
    X = merged_df.drop(columns=['pv_measurement'])
    y = merged_df['pv_measurement']
    
    predictions = []
    models = []
    scores = []
    seeds = range(number_of_models)
    
    for seed in seeds:
        X_train, X_validation, y_train, y_validation = train_test_split(
            X, y, train_size=0.8, random_state=seed)
        
        catboost_model = CatBoostRegressor(
            cat_features=['observed'],
            iterations=10000,
            learning_rate=0.1,
            depth=6,
            loss_function='MAE',
            eval_metric='MAE',
            random_seed=seed,
            verbose=False
        )
        
        catboost_model.fit(X_train, y_train, eval_set=(X_validation, y_validation),
                           use_best_model=True, early_stopping_rounds=200)
        
        score = catboost_model.get_best_score()['validation']['MAE']
        scores.append(score)
        # Print the best validation MAE for the current seed
        print(f"Best validation MAE for seed {seed}: {score}")
        
        
        # Predict using the current model
        preds = catboost_model.predict(x_test)
        predictions.append(preds)
        models.append(catboost_model)
    
    # Average the predictions from all models
    averaged_predictions = np.mean(predictions, axis=0)
    average_score = np.mean(scores, axis = 0)
    
    return averaged_predictions,models, average_score

In [75]:
pred_a1, models_a, avg_a = build_catboost_multiple_seed(merged_a1,x_test_a1,30)
pred_b1, models_b, avg_b = build_catboost_multiple_seed(merged_b1,x_test_b1,30)
pred_c1, models_c, avg_c= build_catboost_multiple_seed(merged_c1,x_test_c1,30)

KeyboardInterrupt: 

In [ ]:
print(avg_a, avg_b, avg_c)

172.9859299746867 24.799131480118685 21.76935013280435


In [ ]:
def create_sub(pred_a,pred_b,pred_c):
    submission = pd.read_csv('sample_submission.csv')
    submission['prediction'] = np.concatenate([pred_a,pred_b,pred_c])
    submission.loc[submission['prediction'] < 0, 'prediction'] = 0
    return submission

In [ ]:
cat_sub = create_sub(pred_a1,pred_b1,pred_c1)

### 2. Autogluon

In [ ]:
#Merge x_train and train for training models
merged_a2 = pd.merge(x_train_a2, train_a, left_on='date_forecast', right_on='time', how='inner')
merged_b2 = pd.merge(x_train_b2, train_b, left_on='date_forecast', right_on='time', how='inner')
merged_c2 = pd.merge(x_train_c2, train_c, left_on='date_forecast', right_on='time', how='inner')

In [ ]:
seed_value = 42  # Replace with your desired seed value
random.seed(seed_value)
np.random.seed(seed_value)

In [ ]:
def build_autogluon(merged_data, time_limit,location):
    merged_df = merged_data.drop(columns=['date_forecast', 'time'])
    
    predictor = TabularPredictor(
        label ='pv_measurement',
        eval_metric= 'mean_absolute_error',
        path = f'AutgluonModels/{location}'
    )

    predictor.fit(
        train_data = merged_df, 
        verbosity = 0,
        presets='best_quality', 
        time_limit= time_limit,
    )
    return predictor

In [ ]:
model_a2 = build_autogluon(merged_a2,3600,'A')
model_b2 = build_autogluon(merged_b2,3600,'B')
model_c2 = build_autogluon(merged_c2,3600,'C')

In [ ]:
pred_a2 = model_a2.predict(x_test_a2)
pred_b2 = model_b2.predict(x_test_b2)
pred_c2 = model_c2.predict(x_test_c2)

gluon_sub = create_sub(pred_a2,pred_b2,pred_c2)

## Part 3: Blend predictions and submit

In [ ]:
#Weighted average of catboost and autogluon model
def weighted_avg(sub1,sub2, w1, w2):
    merged_df = pd.merge(sub1, sub2, on=['id'])
    merged_df['prediction'] = merged_df['prediction_x']*w1 + merged_df['prediction_y']*w2
    final_df = merged_df.drop(columns=['prediction_x', 'prediction_y'])
    final_df.loc[final_df['prediction'] < 8, 'prediction'] = 0
    return final_df

In [ ]:
final_df1 = weighted_avg(cat_sub,gluon_sub,0.7,0.3)
final_df2 = weighted_avg(cat_sub,gluon_sub,0.6,0.4)
final_df3 = weighted_avg(cat_sub,gluon_sub,0.5,0.5)



In [ ]:
final_df2.to_csv('Final_subs/TrainedOverNightShortNotebook.csv', index=False)


In [ ]:
final_df1

,id,prediction
0,0,0.000000
1,1,0.000000
2,2,0.000000
3,3,60.807840
4,4,364.873271
...,...,...
2155,2155,73.996447
2156,2156,42.554606
2157,2157,18.045563
2158,2158,0.000000


In [ ]:
final_df1.to_csv('Final_subs/TrainedOverNightShortNotebook70_30.csv', index=False)
final_df3.to_csv('Final_subs/TrainedOverNightShortNotebook5050.csv', index=False)

OSError: Cannot save file into a non-existent directory: 'TrainedOverNightShortNotebook70'